In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('Shimla_new.csv')
df = df.drop("Unnamed: 0", axis=1)

In [3]:
data = df.copy()  # Create a copy of the DataFrame

month_mapping = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
}

data['Month'] = data['Month'].map(month_mapping)
data['Date'] = pd.to_datetime(data[['Year', 'Month']].assign(day=1))
data.set_index('Date', inplace=True)

In [4]:
# Split your data into training and testing (e.g., use 2010-2018 for training and 2019 for testing)
train_data = data['2010-01-01':'2017-12-01']
test_data = data['2017-01-01':'2019-12-01']

In [5]:
data.head()

,Year,Month,Total Tourists,Category : All,Category : Travel,Category : Shopping,Category : Food and Drink,Category : Auto and Vehicles,Flights to “Shimla”,Trains to “Shimla”,Category : Hotels,Rain,T_Max,T_Min
Date,,,,,,,,,,,,,,
2010-01-01,2010,1,172312,33,42,0,10,0,51,0,40,11.4156,19.98,2.03
2010-02-01,2010,2,161860,29,32,50,16,0,0,0,66,73.3616,24.85,3.66
2010-03-01,2010,3,188961,35,47,51,5,0,81,20,78,25.8435,31.64,8.52
2010-04-01,2010,4,363600,36,47,0,0,0,100,0,100,25.6280,37.60,13.62
2010-05-01,2010,5,369630,41,45,100,12,0,39,44,94,56.6109,38.56,14.89


In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_addons as tfa
from tfa.layers import TemporalFusionTransformer

# Load the dataset
# data = pd.read_csv("your_data.csv")

# Select the relevant columns
data1 = data[['Category : All', 'Category : Travel', 'Category : Shopping', 'Category : Food and Drink',
             'Category : Auto and Vehicles', 'Flights to “Shimla”', 'Trains to “Shimla”',
             'Rain', 'T_Max', 'T_Min']]
data2 = data[['Total Tourists']]

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data1)

# Define the sequence length and split the data into input sequences and target values
sequence_length = 12  # You can adjust this value
X, y = [], []

for i in range(len(data_scaled) - sequence_length):
    X.append(data_scaled[i:i + sequence_length, 1:])  # Use iloc to select features from the second column onwards
    y.append(data2.iloc[i + sequence_length, 0])  # Select the "Total Tourists" column as the target

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train = X[:96]
y_train = y[:96]
X_test = X[84:108]
y_test = y[84:108]

# Create the Temporal Fusion Transformer model
model = Sequential()
model.add(TemporalFusionTransformer(hidden_dim=64, num_heads=4, dropout=0.1))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=32)  # You can adjust the number of epochs and batch size


C:\Users\diksh\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


ModuleNotFoundError: No module named 'tfa'

In [ ]:
  # Use the last data points from the test set as input    
predictions = model.predict(X[72:84])  # Predict the "Total Tourists" for the next time step

In [ ]:
import matplotlib.pyplot as plt

# Inverse transform the actual test data to get it in the original scale

# Plot actual and predicted values
plt.figure(figsize=(12, 6))
plt.plot(y[72:84], label='Actual', color='blue', marker='o')
plt.plot(predictions, label='Predicted', color='red', marker='x')
plt.title('Actual vs. Predicted Time Series Data')
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()
